# **DASHBOARD IMPLEMENTATION**

In [ ]:
from flask import Flask, render_template, request, jsonify
import numpy as np
from stable_baselines3 import DQN, SAC
import gymnasium as gym
from gymnasium import spaces
import os
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)

# Initialize models as None
dqn_model = None
sac_model = None

# Try loading models
try:
    if os.path.exists("dqn_icu.zip"):
        dqn_model = DQN.load("dqn_icu")
        logger.info("Successfully loaded DQN model")
    if os.path.exists("sac_icu.zip"):
        sac_model = SAC.load("sac_icu")
        logger.info("Successfully loaded SAC model")
except Exception as e:
    logger.error(f"Error loading models: {str(e)}")

class ICUEnv(gym.Env):
    def __init__(self):
        super(ICUEnv, self).__init__()
        # Changed observation space to match what DQN expects
        self.observation_space = spaces.Box(
            low=np.array([60, 80, 90, 0, 0]),  # Added 2 extra dimensions
            high=np.array([180, 100, 160, 1, 1]), 
            dtype=np.float32
        )
        self.action_space = spaces.Discrete(3)
        self.state = np.array([100, 95, 120, 0, 0])  # Added padding

    def step(self, action):
        heart_rate, oxygen_level, blood_pressure, _, _ = self.state

        if action == 0:  # Decrease
            heart_rate -= 5
            blood_pressure -= 3
        elif action == 2:  # Increase
            heart_rate += 5
            blood_pressure += 3

        self.state = np.array([
            np.clip(heart_rate, 60, 180),
            np.clip(oxygen_level, 80, 100),
            np.clip(blood_pressure, 90, 160),
            0,  # Padding
            0   # Padding
        ])

        reward = -abs(heart_rate - 100) - abs(oxygen_level - 95) - abs(blood_pressure - 120)
        return self.state, reward, False, False, {}

    def reset(self, seed=None):
        self.state = np.array([100, 95, 120, 0, 0])  # Added padding
        return self.state, {}

class ICUEnvContinuous(gym.Env):
    def __init__(self):
        super().__init__()
        self.observation_space = spaces.Box(
            low=np.array([60, 80, 90]), 
            high=np.array([180, 100, 160]), 
            dtype=np.float32
        )
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)
        self.state = np.array([100, 95, 120])

    def step(self, action):
        heart_rate, oxygen_level, blood_pressure = self.state
        change = float(action[0]) * 5
        
        self.state = np.array([
            np.clip(heart_rate + change, 60, 180),
            np.clip(oxygen_level, 80, 100),
            np.clip(blood_pressure + change, 90, 160)
        ])

        reward = -abs(self.state[0] - 100) - abs(self.state[1] - 95) - abs(self.state[2] - 120)
        return self.state, reward, False, False, {}

    def reset(self, seed=None):
        self.state = np.array([100, 95, 120])
        return self.state, {}

# Initialize environments
discrete_env = ICUEnv()
continuous_env = ICUEnvContinuous()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        if not data:
            return jsonify({'error': 'No data received'}), 400

        model_type = data.get('model_type')
        
        if model_type == 'dqn':
            if dqn_model is None:
                return jsonify({'error': 'DQN model not loaded'}), 503
            # For DQN: 5D observation
            current_state = np.array([
                float(data.get('heart_rate', 100)),
                float(data.get('oxygen_level', 95)),
                float(data.get('blood_pressure', 120)),
                float(data.get('resp_rate', 16))/40,  # Normalized
                float(data.get('gcs', 15))/15          # Normalized
            ], dtype=np.float32)
            action, _ = dqn_model.predict(current_state)
            action_name = ["Decrease Medication", "Maintain", "Increase Medication"][int(action)]
            return jsonify({'action': str(action), 'action_name': action_name})

        elif model_type == 'sac':
            if sac_model is None:
                return jsonify({'error': 'SAC model not loaded'}), 503
            # For SAC: 3D observation
            current_state = np.array([
                float(data.get('heart_rate', 100)),
                float(data.get('oxygen_level', 95)),
                float(data.get('blood_pressure', 120))
            ], dtype=np.float32)
            action, _ = sac_model.predict(current_state)
            action_name = f"Adjust Medication by {float(action[0]):.2f}"
            return jsonify({'action': str(action[0]), 'action_name': action_name})

        return jsonify({'error': 'Invalid model type'}), 400

    except Exception as e:
        logger.error(f"Prediction error: {str(e)}")
        return jsonify({'error': str(e)}), 500

@app.route('/simulate', methods=['POST'])
def simulate():
    try:
        data = request.get_json()
        model_type = data.get('model_type')
        action = data.get('action')
        
        if model_type == 'dqn':
            discrete_env.state = np.array([
                float(data.get('heart_rate', 100)),
                float(data.get('oxygen_level', 95)),
                float(data.get('blood_pressure', 120)),
                0,  # Padding
                0   # Padding
            ])
            state, _, _, _, _ = discrete_env.step(int(action))
            # Return only the first 3 values to frontend
            return jsonify({
                'heart_rate': float(state[0]),
                'oxygen_level': float(state[1]),
                'blood_pressure': float(state[2])
            })
        else:
            continuous_env.state = np.array([
                float(data.get('heart_rate', 100)),
                float(data.get('oxygen_level', 95)),
                float(data.get('blood_pressure', 120))
            ])
            state, _, _, _, _ = continuous_env.step(np.array([float(action)]))
            return jsonify({
                'heart_rate': float(state[0]),
                'oxygen_level': float(state[1]),
                'blood_pressure': float(state[2])
            })

    except Exception as e:
        logger.error(f"Simulation error: {str(e)}")
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)

INFO:__main__:Successfully loaded DQN model
INFO:__main__:Successfully loaded SAC model


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.29.244:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 12:48:14] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 12:48:15] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 12:48:38] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 12:48:54] "POST /simulate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 12:49:20] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2025 12:49:35] "POST /simulate HTTP/1.1" 200 -
